In [ ]:
from functions import transform_functions as traf
from functions import preprossesing as prep
import numpy as np

filename = "running-example"
ocel, act_dict, cust_dict = traf.prepare_ocel(filename)
col = ["Items","Products","Orders",]
# for c in col:
#     print(c)
#     print(np.unique([len(t) for t in ocel[c]]))
print(act_dict)
print(cust_dict)
# display(ocel.head())
drops_col_order = ["weight","price","Event_ID",'Products']
ocel_orders = traf.flatten(ocel,'Orders')
enriched_log, single_log = traf.gen_flatted_comp_csv(ocel_orders,'Orders',drops_col_order)

In [ ]:
enriched_log

In [ ]:
np.random.seed(42)
from operator import itemgetter
import pandas as pd
from datetime import datetime
lastcase = ''
line = ''
firstLine = True
lines = []
Ptimeseqs = []
Ptimeseqs2 = []
Ptimeseqs3 = []
Ptimeseqs4 = []

Ptimes = []
Ptimes2 = []
Ptimes3 = []
Ptimes4 = []

numlines = 0
Pcasestarttime = None
Plasteventtime = None

for index, row in enriched_log.iterrows():
    t = row['Timestamp']

    if row['Case_ID'] != lastcase:
        Stime = t
        Pcasestarttime = t
        Plasteventtime = t
        lastcase = row['Case_ID']
        if not firstLine:
            lines.append(line)
            Ptimeseqs.append(Ptimes)
            Ptimeseqs2.append(Ptimes2)
            Ptimeseqs3.append(Ptimes3)
            Ptimeseqs4.append(Ptimes4)

        line = ''
        Ptimes = []
        Ptimes2 = []
        Ptimes3 = []
        Ptimes4 = []

        numlines += 1

    line += chr(int(row['Activity']) + ascii_offset)
    Ptimesincelastevent = t - Plasteventtime
    Ptimesincecasestart = t - Pcasestarttime
    midnight = t.replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = t - midnight
    Ptimediff = 86400 * Ptimesincelastevent.days + Ptimesincelastevent.seconds
    Ptimediff2 = 86400 * Ptimesincecasestart.days + Ptimesincecasestart.seconds
    Ptimediff3 = timesincemidnight.seconds
    Ptimediff4 = t.weekday()
    Ptimes.append(Ptimediff)
    Ptimes2.append(Ptimediff2)
    Ptimes3.append(Ptimediff3)
    Ptimes4.append(Ptimediff4)

    Plasteventtime = t
    firstLine = False

# Add last case
lines.append(line)
Ptimeseqs.append(Ptimes)
Ptimeseqs2.append(Ptimes2)
Ptimeseqs3.append(Ptimes3)
Ptimeseqs4.append(Ptimes4)

PtimeseqsF = []
for seq in Ptimeseqs2:
    l = []
    for t in seq:
        l.append(seq[-1] - t)
    PtimeseqsF.append(l)

numlines += 1
########################################

divisor = np.mean([item for sublist in Ptimeseqs for item in sublist])  # average time between events
print('divisor: {}'.format(divisor))
divisor2 = np.mean(
    [item for sublist in Ptimeseqs2 for item in sublist])  # average time between current and first events
print('divisor2: {}'.format(divisor2))
divisorTR = np.mean(
    [item for sublist in PtimeseqsF for item in sublist])  # average time instance remaining
print('divisor2: {}'.format(divisorTR))
#########################################################################################################

# separate training data into 3 parts
np.random.seed(42)
indices = np.random.permutation(numlines - 1)
elems_per_fold = int(round(numlines / 3))

idx1 = indices[:elems_per_fold]
idx2 = indices[elems_per_fold:2 * elems_per_fold]
idx3 = indices[2 * elems_per_fold:]

In [ ]:

# elems_per_fold = int(round(numlines / 3))
import csv
from numpy.compat import unicode



fold1 = list(itemgetter(*idx1)(lines))
fold1_t = list(itemgetter(*idx1)(Ptimeseqs))
fold1_t2 = list(itemgetter(*idx1)(Ptimeseqs2))
fold1_t3 = list(itemgetter(*idx1)(Ptimeseqs3))
fold1_t4 = list(itemgetter(*idx1)(Ptimeseqs4))
fold1_t5 = list(itemgetter(*idx1)(PtimeseqsF))
with open('../output_files/folds/fold1.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold1, fold1_t):
        spamwriter.writerow([unicode(s) + '#{}'.format(t) for s, t in zip(row, timeseq)])

fold2 = list(itemgetter(*idx2)(lines))
fold2_t = list(itemgetter(*idx2)(Ptimeseqs))
fold2_t2 = list(itemgetter(*idx2)(Ptimeseqs2))
fold2_t3 = list(itemgetter(*idx2)(Ptimeseqs3))
fold2_t4 = list(itemgetter(*idx2)(Ptimeseqs4))
fold2_t5 = list(itemgetter(*idx2)(PtimeseqsF))
with open('../output_files/folds/fold2.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold2, fold2_t):
        spamwriter.writerow([unicode(s) + '#{}'.format(t) for s, t in zip(row, timeseq)])

fold3 = list(itemgetter(*idx3)(lines))
fold3_t = list(itemgetter(*idx3)(Ptimeseqs))
fold3_t2 = list(itemgetter(*idx3)(Ptimeseqs2))
fold3_t3 = list(itemgetter(*idx3)(Ptimeseqs3))
fold3_t4 = list(itemgetter(*idx3)(Ptimeseqs4))
fold3_t5 = list(itemgetter(*idx3)(PtimeseqsF))
with open('../output_files/folds/fold3.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold3, fold3_t):
        spamwriter.writerow(
            [unicode(s).encode("utf-8") + '#{}'.format(t).encode('utf-8') for s, t in zip(row, timeseq)])

lines = fold1 + fold2
lines_t = fold1_t + fold2_t
lines_t2 = fold1_t2 + fold2_t2
lines_t3 = fold1_t3 + fold2_t3
lines_t4 = fold1_t4 + fold2_t4
lines_t5 = fold1_t5 + fold2_t5

step = 1
sentences = []
softness = 0
next_chars = []
lines = list(map(lambda x: x + '!', lines))  # put delimiter symbol
maxlen = max(map(lambda x: len(x), lines))  # find maximum line size

# next lines here to get all possible characters for events and annotate them with numbers
# chars = map(lambda x: set(x),lines)
chars = list(set().union(*map(lambda x: set(x), lines)))
chars.sort()
target_chars = copy.copy(chars)
chars.remove('!')
print('total chars: {}, target chars: {}'.format(len(chars), len(target_chars)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i, c) for i, c in enumerate(target_chars))
print(indices_char)

In [ ]:
#### comparing if product and item match but is to hard to handle them

t1 = [len(t) for t in ocel['Items']]
t2 = [len(t) for t in ocel['Products']]
t3 = [a!=b for a, b in zip(t1, t2)]
print(ocel.size)
print(sum(t3))
print(sum(t3)/ocel.size)
print(len(t3) - sum(t3))
# display(ocel[t3])
# display(ocel[t3].explode('Products').groupby(['Products']).size().sort_values())
oceltemp = ocel[t3].copy()
t1 = [len(t) for t in oceltemp['Items']]
t2 = [len(t) for t in oceltemp['Products']]
t3 = [a!=b for a, b in zip(t1, t2)]
print(sum(t3))
df_agg = ocel.copy()
df_agg.Items = df_agg.Items.apply(len)
df_agg.Orders = df_agg.Orders.apply(len)
df_agg.Products = df_agg.Products.apply(len)
df_agg = df_agg.groupby('Packages').sum(numeric_only=True)
display(df_agg)